Use decision trees to prepare a model on fraud data 
treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("D:\\DATA SCIENCE\\DATA SCIENCE ASSIGNMENTS\\Assignments\\Decision Tree\\Fraud_check.csv")
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [4]:
data.rename(columns={'Undergrad':'Ugrad','Marital.Status':'Mstatus','Taxable.Income':'Taxinc','City.Population':'Citypopu','Work.Experience':'Wexp'},inplace=True)

In [5]:
data.head()

,Ugrad,Mstatus,Taxinc,Citypopu,Wexp,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [6]:
df=data.copy()

In [7]:
df['Ugrad'].replace({'NO':0,'YES':1},inplace=True)
df['Mstatus'].replace({'Single':0,'Married':1,'Divorced':2},inplace=True)
df['Urban'].replace({'NO':0,'YES':1},inplace=True)

In [8]:
df

,Ugrad,Mstatus,Taxinc,Citypopu,Wexp,Urban
0,0,0,68833,50047,10,1
1,1,2,33700,134075,18,1
2,0,1,36925,160205,30,1
3,1,0,50190,193264,15,1
4,0,1,81002,27533,28,0
...,...,...,...,...,...,...
595,1,2,76340,39492,7,1
596,1,2,69967,55369,2,1
597,0,2,47334,154058,0,1
598,1,1,98592,180083,17,0


In [9]:
y=df['Taxinc']
Y=[0 if b<=30000 else 1 for b in y]
Y=pd.DataFrame(Y)
Y.value_counts()   #inbalance Y

1    476
0    124
dtype: int64

In [10]:
df1=df.copy()

In [11]:
X=df1.drop('Taxinc',axis=1)
X

,Ugrad,Mstatus,Citypopu,Wexp,Urban
0,0,0,50047,10,1
1,1,2,134075,18,1
2,0,1,160205,30,1
3,1,0,193264,15,1
4,0,1,27533,28,0
...,...,...,...,...,...
595,1,2,39492,7,1
596,1,2,55369,2,1
597,0,2,154058,0,1
598,1,1,180083,17,0


## Build Decision Tree Classifier Entropy Criterion  (C5.0)

In [12]:
#model validation technique(train test split)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.30)

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [14]:
model_entropy=DecisionTreeClassifier(criterion='entropy',max_depth=3)
model_entropy.fit(x_train,y_train)
y_pred=model_entropy.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.79      0.98      0.88       143

    accuracy                           0.78       180
   macro avg       0.40      0.49      0.44       180
weighted avg       0.63      0.78      0.70       180



In [15]:
#model validation technique(k-fold)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [16]:
kfold=KFold(n_splits=8)
results=cross_val_score(model_entropy,X,Y,cv=kfold)
results.mean()

0.79

In [17]:
#Ensemble technique(Bagging)
from sklearn.ensemble import BaggingClassifier

In [26]:
model1=BaggingClassifier(base_estimator=model_entropy,n_estimators=4)
model1.fit(x_train,y_train)
y_pred1=model1.predict(x_test)
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.79      1.00      0.89       143

    accuracy                           0.79       180
   macro avg       0.40      0.50      0.44       180
weighted avg       0.63      0.79      0.70       180



D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
#bagging accuracy using k-fold
kfold=KFold(n_splits=11)
results=cross_val_score(model1,X,Y.values.ravel(),cv=kfold)
results.mean()

0.7918885827976737

In [28]:
#ensemble technique(AdaBoost)
from sklearn.ensemble import AdaBoostClassifier

In [29]:
model2=AdaBoostClassifier(n_estimators=10)
model2.fit(x_train,y_train)
y_pred2=model2.predict(x_test)
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.79      1.00      0.89       143

    accuracy                           0.79       180
   macro avg       0.40      0.50      0.44       180
weighted avg       0.63      0.79      0.70       180



D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
#adaboost accuracy using k-fold
kfold=KFold(n_splits=8)
results=cross_val_score(model2,X,Y.values.ravel(),cv=kfold)
results.mean()

0.7933333333333333

## Build Decision Tree Classifier gini Criterion  (CART)

In [31]:
#model validation technique(train test split)
X_TRAIN,X_TEST,Y_TRAIN,Y_TEST=train_test_split(X,Y,test_size=0.25)

In [32]:
model_gini=DecisionTreeClassifier(criterion='gini',max_depth=3)
model_gini.fit(X_TRAIN,Y_TRAIN)
Y_PRED=model_gini.predict(X_TEST)
print(classification_report(Y_TEST,Y_PRED))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.79      0.98      0.88       119

    accuracy                           0.78       150
   macro avg       0.40      0.49      0.44       150
weighted avg       0.63      0.78      0.70       150



In [33]:
#model validation technique(k-fold)
kfold=KFold(n_splits=9)
results=cross_val_score(model_gini,X,Y,cv=kfold)
results.mean()

0.7886074677119452

In [34]:
#Ensemble technique(Bagging)
model3=BaggingClassifier(base_estimator=model_gini,n_estimators=10)
model3.fit(X_TRAIN,Y_TRAIN)
Y_PRED3=model3.predict(X_TEST)
print(classification_report(Y_TEST,Y_PRED3))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.79      0.97      0.87       119

    accuracy                           0.77       150
   macro avg       0.39      0.48      0.43       150
weighted avg       0.62      0.77      0.69       150



D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [35]:
#bagging accuracy using k-fold
kfold=KFold(n_splits=6)
results=cross_val_score(model3,X,Y.values.ravel(),cv=kfold)
results.mean()

0.7933333333333333

In [36]:
#ensemble technique(AdaBoost)
model4=AdaBoostClassifier(n_estimators=10)
model4.fit(X_TRAIN,Y_TRAIN)
Y_PRED4=model4.predict(X_TEST)
print(classification_report(Y_TEST,Y_PRED4))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.79      1.00      0.88       119

    accuracy                           0.79       150
   macro avg       0.40      0.50      0.44       150
weighted avg       0.63      0.79      0.70       150



D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
#adaboost accuracy using k-fold
kfold=KFold(n_splits=8)
results=cross_val_score(model4,X,Y.values.ravel(),cv=kfold)
results.mean()

0.7933333333333333

In [38]:
#ensemble technique(stacking)
from sklearn.ensemble import VotingClassifier

In [39]:
#make a list whose accuracy max
estimators=[]
estimators.append(('DTclassifierC5.0entropy',model_entropy))
estimators.append(('DTclassifierC5.0entropy(Bagging)',model1))
estimators.append(('DTclassifierC5.0entropy(AdaBoost)',model2))
estimators.append(('DTclassifierCARTgini',model_gini))
estimators.append(('DTclassifierCARTgini(Bagging)',model3))
estimators.append(('DTclassifierCARTgini(AdaBoost)',model4))

In [41]:
#model validation technique(train test split)
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=0.33)
stacking=VotingClassifier(estimators)
stacking.fit(train_x,train_y)
stacking.score(test_x,test_y)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.797979797979798

In [48]:
#Model validation technique(k-fold)
kfold=KFold(n_splits=9)
results=cross_val_score(stacking,X,Y.values.ravel(),cv=kfold)
results.mean()

0.791899090406553